# Team Member:


107062514 賴鵬仁

107062616 傅品捷

107065513 姚定嘉

### 1. Preprocessing: Cut Words

In [ ]:
%matplotlib inline

import pandas as pd

NUM_PROGRAM = 8
programs = []


for i in range(1, NUM_PROGRAM+1):
    program = pd.read_csv('Program0%d.csv' % (i))
    
    print('Program %d' % (i))
    print('Episodes: %d' % (len(program)))
    print(program.columns)
    print()
    
    print(program.loc[:1]['Content'])
    print()
    
    programs.append(program)


questions = pd.read_csv('Question.csv')

print('Question')
print('Episodes: %d' % (len(questions)))
print(questions.columns)
print()

print(questions.loc[:2]['Question'])
print()

for i in range(6):
    print(questions.loc[:2]['Option%d' % (i)])
    print()

    
import jieba

jieba.set_dictionary('big5_dict.txt')
example_str = '我討厭吃蘋果'
cut_example_str = jieba.lcut(example_str)
print(cut_example_str)

def jieba_lines(lines):
    cut_lines = []
    
    for line in lines:
        cut_line = jieba.lcut(line)
        cut_lines.append(cut_line)
    
    return cut_lines

cut_programs = []

for program in programs:
    episodes = len(program)
    cut_program = []
    
    for episode in range(episodes):
        lines = program.loc[episode]['Content'].split('\n')
        cut_program.append(jieba_lines(lines))
    
    cut_programs.append(cut_program)
    
print("%d programs" % len(cut_programs))
print("%d episodes in program 0" % len(cut_programs[0]))
print("%d lines in first episode of program 0" % len(cut_programs[0][0]))

print()
print("first 3 lines in 1st episode of program 0: ")
print(cut_programs[0][0][:3])

按照notebook的做法，用 jieba 將每一句切成以詞為單位的 list 

In [ ]:
cut_questions = []
n = len(questions)

for i in range(n):
    cut_question = []
    lines = questions.loc[i]['Question'].split('\n')
    cut_question.append(jieba_lines(lines))
    
    for j in range(6):
        line = questions.loc[i]['Option%d' % (j)]
        cut_question.append(jieba.lcut(line))
    
    cut_questions.append(cut_question)
print("%d questions" % len(cut_questions))
print(len(cut_questions[0]))

# 1 question
print(cut_questions[0][0])

# 6 optional reponses
for i in range(1, 7):
    print(cut_questions[0][i])

import numpy as np

np.save('cut_Programs.npy', cut_programs)
np.save('cut_Questions.npy', cut_questions)

cut_programs = np.load('cut_Programs.npy')
cut_Question = np.load('cut_Questions.npy')

將其分別存成 cut_Programs.npy 跟 cut_Questions.npy

### 2. Word Dictionary & Out-of-Vocabulary

In [ ]:
word_dict = dict()
def add_word_dict(w):
    if not w in word_dict:
        word_dict[w] = 1
    else:
        word_dict[w] += 1
for program in cut_programs:
    for lines in program:
        for line in lines:
            for w in line:
                add_word_dict(w)
for question in cut_questions:
    lines = question[0]
    for line in lines:
        for w in line:
            add_word_dict(w)
    
    for i in range(1, 7):
        line = question[i]
        for w in line:
            add_word_dict(w)
import operator

word_dict = sorted(word_dict.items(), key=operator.itemgetter(1), reverse=True)
print("Total %d words in word_dict" % len(word_dict))

In [ ]:
VOC_SIZE = 75000
VOC_START = 1

voc_dict = word_dict[VOC_START:VOC_START+VOC_SIZE]
print(voc_dict[:10])
print()
print("Total %d words in voc_dict" % len(voc_dict))
np.save('voc_dict.npy', voc_dict)

voc_dict = np.load('voc_dict.npy')

In [ ]:
cut_programs = np.load('cut_Programs.npy')
cut_questions = np.load('cut_Questions.npy')

program_out_a = open('all_refined.txt', 'w', encoding="utf-8") 
program_out_p = open('programs_refined.txt', 'w', encoding="utf-8")
program_out_q = open('questions_refined.txt', 'w', encoding="utf-8")


stopword_set = set()
stopword_set.add('\u3000')
stopword_set.add('\t')
stopword_set.add(' ')
stopword_set.add('...')
stopword_set.add('「')
stopword_set.add('」')


cut_programs_refined = []
for program in cut_programs:
    episode_refined = []
    for episode in program:
        line_refined = []
        for line in episode:
            ex_line = ''
            for w in line:
                if (w in voc_dict) and (w not in stopword_set):
                    ex_line += w + ' '
                    line_refined.append(w)
            program_out_a.write(ex_line+'\n')
            program_out_p.write(ex_line+'\n')
            
    episode_refined.append(line_refined)
    
cut_programs_refined.append(episode_refined)


for question in cut_questions:
    for i, line in enumerate(question):
        if (i == 0):
            ex_line = ''
            for qline in question[i]:
                for w in qline:
                    if (w in voc_dict) and (w not in stopword_set):
                        ex_line += w + ' '
            program_out_a.write(ex_line+'\n')
            program_out_q.write(ex_line+'\n')
        else:
            ex_line = ''
            for w in line:
                if (w in voc_dict) and (w not in stopword_set):
                    ex_line += w + ' '
            program_out_a.write(ex_line+'\n')
            program_out_q.write(ex_line+'\n')
            
program_out_a.close()
program_out_p.close()
program_out_q.close()

There are many words after cutting, but not all of them are useful. The words too common or too rare can not give us information but may introduce noise. We count the the number of occurrence for each word and remove useless one.

此外，我們在分析句子的時候發現，有些句子中有包含 "\t"、"whitespace"......等，所以我們將這些不重要的字元過濾掉，並將處理完的資料存起來以利後續使用。

### 3. Word2vec Model

In [ ]:
from gensim.models import word2vec

sentences = word2vec.Text8Corpus('all_refined.txt')

model = word2vec.Word2Vec(sentences, hs=1, negative=0.0001, min_count=1,size = 600)

model.save('word2vec_model.bin')

接下來我們就用 word2vec 去 generate 每一個詞的向量。在這邊我們tune了很多參數，包括(windows, size, sg, hs, negative, min_count...等)。我們在一開始使用size=100，發現效果不佳，經過一番測試，發現size=600的表現最好。
windows的部分我們也發現用它default的參數會得到實驗中最好的效果，因此我們windows的部分並沒有調整。而我們min_count設為1的目的是希望可以考慮到只出現過一次的詞，我們認為這些詞在這段文字中也相當重要。

在train完word2vec model之後也會把model存起來以利後續使用。

### 4. Generate Training Dataset 

In [ ]:
from gensim.models import word2vec

def read_txt(filename):
    result = []
    with open(filename, 'r', encoding="utf-8") as inFile:
        for line in inFile:
            result.append(line.strip('\n'))
    return result
        
programs_refined = read_txt('programs_refined.txt')
questions_refined = read_txt('questions_refined.txt')
all_refined = read_txt('all_refined.txt')

word2vec_model = word2vec.Word2Vec.load('word2vec_model.bin')

In [ ]:
import random

NUM_TRAIN = 500000
TRAIN_VALID_RATE = 0.7
def generate_training_data():
    Xs, Ys = [], []
    
    for i in range(NUM_TRAIN):
        pos_or_neg = random.randint(0, 1)
        
        if pos_or_neg==1:
            line_id = random.randint(1, len(programs_refined)-1)
            
            L1 = programs_refined[line_id].split(' ')[:-1]
            L2 = programs_refined[line_id+1].split(' ')[:-1]
            L3 = programs_refined[line_id-1].split(' ')[:-1]
            
            if (L1 and L2 and L3) :
                sim1 = word2vec_model.wv.n_similarity(L1, L2)
                sim2 = word2vec_model.wv.n_similarity(L1, L3)

                if (sim1 > sim2):
                    Xs.append(programs_refined[line_id])
                    Xs.append(programs_refined[line_id+1])
                else:
                    Xs.append(programs_refined[line_id-1])
                    Xs.append(programs_refined[line_id])

                Ys.append(1)
            
        else:
            
            first_line_id = random.randint(0, len(programs_refined)-1)
            second_line_id = random.randint(0, len(programs_refined)-1)
            
            L1 = programs_refined[first_line_id].split(' ')[:-1]
            L2 = programs_refined[second_line_id].split(' ')[:-1]
            
            if L1 and L2:
                Xs.append(programs_refined[first_line_id])
                Xs.append(programs_refined[second_line_id])
                Ys.append(0)
                    
    return Xs, Ys

X, Y = generate_training_data()

我們將train好的word2vec model 載入，將之前做好的資料也一起載入。

而我們 generate training data的方法是，我們會 random generate 出postive training data 以及 negative training data。
Positive 的部分，我們隨機選一句並去觀察他的上一句與下一句，因為隨機挑出的句子可能會是該episode或是該串對話串的最後一句或是第一句。因此，我們分別比較該句子與其上下的 cosine similarity，取其cosine similarity 高的兩句作為 training data set 的Ｘ，並令其對應的Ｙ為 1。
Negative 的部分，我們則就整個 programs 隨機選兩句，作為 training data set 的 Ｘ，並令其對應的Ｙ為 0。

### 4. TF-IDF Model

In [ ]:
for question in questions_refined:
    X.append(question)

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(ngram_range=(3,4))
tfidf.fit(X)
#doc_tf = tfidf.transform(X).toarray()
doc_tf_sparse = tfidf.transform(X)

TF-IDF使用ngram_range = (3, 4)。

### 5. Feature Extraction

In [ ]:
import numpy as np

doc_vec = []

i = 0
for sentence, tf_idf in zip(X, doc_tf_sparse):
    words = sentence.split(' ')[:-1]
    vec = np.zeros(shape=(600,), dtype=float)
    
    for w in words:
        try:
            vec = np.add(vec, word2vec_model.wv[w]*tf_idf[i, tfidf.vocabulary_[w]]*100)
        except KeyError:
            vec = np.add(vec, word2vec_model.wv[w]*0.1)
    
    i += 1
    doc_vec.append(vec)

q_start = len(X) - len(questions_refined)

programs_vec = doc_vec[:q_start]
questions_vec = doc_vec[q_start:]

在取完 TF-IDF 後，我們將每個句子做成向量。我們的方法是，將每一句裡面包含的詞從 word2vec model 中取出該詞的向量，並以他對應的 TF-IDF 值當作weight，最後 sum 起來得到該句子的最終向量。

In [ ]:
X_ = []

def cosine_sim(a, b):
    return (np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))) * np.array([1])

for _id in range(int(len(programs_vec) / 2)):
    X_.append(cosine_sim(programs_vec[_id*2], programs_vec[_id*2+1]))

X_ = np.stack(X_, axis=0)
y = np.array(Y)

最後，我們將 training data 中上下兩句的cosine similarity當成餵進model的input。

### 6. Sklearn Model Training

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

X_train, X_valid, y_train, y_valid = train_test_split(X_, y, test_size=0.3, random_state=0)
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_valid_std = sc.transform(X_valid)

from sklearn.linear_model import Lasso

las = Lasso(alpha=0.002)

las.fit(X_train_std, y_train)
y_pred_las = las.predict(X_valid_std)

print ("MSE: %.4f" % (mean_squared_error(y_valid, y_pred_las)))

MSE: 0.1729

這裡使用Lasso，因為問題是六選一的選擇題，我們希望model輸出的是每個選項的分數，並挑選分數最高的選項當成prediction的結果；又，我們期望model有較好的generalizability，所以選擇Lasso。

經過實驗後，我們發現比起 learning model，feature engineering 相對重要許多。一開始我們的使用 feature hashing 得到的 accuracy 僅僅只有 0.16左右，但再加上 word2vec 與 TF-IDF 後 accuracy 就達到了0.62，接著在 tune 完 word2vec 以及 TF-IDF 的參數後，accuracy 可以達到 0.66。

### 7. Generate Testing set Prediction

In [ ]:
import sys

q_test = []
for q_id in range(500):
    question = questions_vec[q_id*7]
    answer = 0
    sim_max = sys.float_info.min
    for i in range(1,7):
        ans = questions_vec[q_id*7 + i]
        set_ = cosine_sim(question, ans)
        try:
            result = las.predict(set_.reshape(1,-1))
        except ValueError:
            continue
        if result > sim_max:
            answer = i-1
            sim_max = result
        
    q_test.append((q_id, answer))

In [ ]:
import pandas as pd

df = pd.DataFrame(data=q_test, columns=['Id', 'Answer']).to_csv('Answer.csv', index=False)

## Conclusion 

聽完前三名的組別分享，其中兩組都使用了BiLSTM，扣除掉用NN model 外 ，大家的共通點都是有利用更好的切字library 及 比word2vec 更好的fasttext model。在training data 方面，雖然我們有做額外的處理，但都和前幾組相比，做的不夠細膩也考慮的比較不周全。第一名的組別沒用到NN 就可以達到很高的準確率，可見feature engineering 和 train data的生成都是非常的關鍵。 這次比賽過程與最後別人的分享都學到了許多，也期待之後比賽的表現能更進步。謝謝老師與助教!